In [ ]:

from photutils.aperture import EllipticalAperture, EllipticalAnnulus, CircularAperture, CircularAnnulus, aperture_photometry 
from photutils.detection import DAOStarFinder
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from scipy.stats import iqr
import numpy as np
import os

# PSF Fitting
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm
from photutils.background import MADStdBackgroundRMS, MMMBackground
from photutils.detection import IRAFStarFinder
from photutils.psf import DAOGroup, IntegratedGaussianPRF, BasicPSFPhotometry


# Settings
Change these to change which image is being reduced and analysed.

In [ ]:
# Image Conditions 
data_path =  os.path.join("data", "480") # Where images are stored
image_path = os.path.join("m81", "M81_Halpha_20230419_055757.fits")
#image_path = os.path.join("m82", "M82_Halpha_20230419_050034.fits")
#image_path = os.path.join("ngc2976", "NGC 2976_Halpha_20230419_040046.fits")
filter = "halpha" # The filter used (must match the flat folder)
wavelength = 656.46E-9 # Wavelength of the filter (in meters)
exposure = 1800

# Target object info
distance = 3.30 * 3.0856776E+22 # Distance to the target (in meters)

# Plotting Settings
log_vmin = 60
log_vmax = 600
color_map = "inferno"

# Aperture Photometry Settings
aperture_position = (1525, 1525)
aperture_a = 450
aperture_b = 650
aperture_theta = -0.5
annulus_inner = 1.2
annulus_outer = 1.8

# CCD Characteristics
gain = 1.5
quantum_efficiency = 0.7 # How many electrons are counted per photon that hits

# FITS Reader

In [ ]:
def get_fits_data(path):
    '''Returns the data and header for every image found in a folder path.'''

    isFolder = False if path.endswith(".fits") else True

    # If this is just a single file, only get its data
    file_paths = []
    if not isFolder: 
        file_paths.append(path)

    # If the path is to a folder, find all files in the folder
    else:
        for (path, dirnames, filenames) in os.walk(path):
            file_paths.extend(os.path.join(path, name) for name in filenames)
    
    # Gets all the fits data and headers from the found files
    data = []
    headers = []
    for file in file_paths:
        data.append(fits.getdata(file))
        headers.append(fits.getheader(file))
    
    # If its a folder, return the data and headers for all found files
    if isFolder: return data, headers

    # If its a single file, return only the first element
    else: return data[0], headers[0]

# Calibration Functions

In [ ]:
def get_master_bias(biases):
    '''Finds the median value for each pixel from a set of biases.'''
    return np.median(biases, axis=0)

def get_master_dark(darks, exposure, bias):
    '''
    Subtracts the bias and finds the median to create a master dark.
    Returns a scaled master dark with per-second values.
    '''
    master_dark = (np.median(darks, axis=0) - bias).clip(min=0)
    return master_dark / exposure

def get_master_flat(flats, flat_headers, dark, bias):
    '''
    Subtracts the bias and dark from each flat, and averages them to create a master flat.
    Returned flat is normalized with 1 being the median valued pixel.
    '''
    exposure = flat_headers[0]['Exposure']
    flat = (flats[0] - (bias + (dark * exposure))) / len(flats)
    for i in range(1, len(flats)):
        exposure = flat_headers[i]['Exposure']
        flat += (flats[i] - (bias + (dark * exposure))) / len(flats)
    return flat / np.median(flat)

# Grabbing Calibrations

In [ ]:
def create_masters(path, filter):

    biases, bias_headers = get_fits_data(os.path.join(path, "biases"))
    master_bias = get_master_bias(biases)

    darks, dark_headers = get_fits_data(os.path.join(path, "darks"))
    master_dark = get_master_dark(darks, 1800, master_bias)

    flats, flat_headers = get_fits_data(os.path.join(path, "flats", filter))
    master_flat = get_master_flat(flats, flat_headers, master_dark, master_bias)

    return master_bias, master_dark, master_flat

In [ ]:
def correct_image(image, bias, dark, flat):
    debiased = (image - bias).clip(min=0)
    dedarked = (debiased - dark * 1800).clip(min=0)
    flat_corrected = dedarked / flat
    return debiased, dedarked, flat_corrected

# Plotting

In [ ]:
"""def plot_fits(image, title, secondary_plot=None, secondary_title="", my_vmin=None, my_vmax=None, save_fig=False):

    if my_vmin == None: my_vmin = np.min(image)    # set vmin to min data value if none was specified
    if my_vmax == None: my_vmax = np.max(image)    # set vmax to max data value if noe was specified
    
    #median = np.mean(image.flatten())
    #image_iqr = iqr(image.flatten())
    #print(f"Median: {median}, IQR: {image_iqr}")
    
    fig, ax = plt.subplot_mosaic(
        '''
        AAB
        AAC
        ''',
        figsize = (12, 6)#, constrained_layout = True
    )

    ax['A'].imshow(image, norm=LogNorm(vmin=my_vmin, vmax=my_vmax), cmap=color_map)#,cmap = plt.cm.viridis)
    ax['A'].set_title(title);
    ax['A'].grid(False)

    if secondary_plot is not None:
        ax['B'].imshow(secondary_plot, cmap=color_map, norm=LogNorm())
        ax['B'].set_title(secondary_title)
    
    ax['C'].set_xlabel("Pixel Values")
    ax['C'].set_ylabel("Log(count)")
    
    ax['C'].hist(image.flatten(), color="red", bins=500, log=True);
    ax['C'].set_xlim(0, 65536)
    #ax['C'].set_ylim(0, 10)
    
    # save image
    #if save_fig:    fig.savefig(image_name);

    plt.show();"""

'def plot_fits(image, title, secondary_plot=None, secondary_title="", my_vmin=None, my_vmax=None, save_fig=False):\n\n    if my_vmin == None: my_vmin = np.min(image)    # set vmin to min data value if none was specified\n    if my_vmax == None: my_vmax = np.max(image)    # set vmax to max data value if noe was specified\n    \n    #median = np.mean(image.flatten())\n    #image_iqr = iqr(image.flatten())\n    #print(f"Median: {median}, IQR: {image_iqr}")\n    \n    fig, ax = plt.subplot_mosaic(\n        \'\'\'\n        AAB\n        AAC\n        \'\'\',\n        figsize = (12, 6)#, constrained_layout = True\n    )\n\n    ax[\'A\'].imshow(image, norm=LogNorm(vmin=my_vmin, vmax=my_vmax), cmap=color_map)#,cmap = plt.cm.viridis)\n    ax[\'A\'].set_title(title);\n    ax[\'A\'].grid(False)\n\n    if secondary_plot is not None:\n        ax[\'B\'].imshow(secondary_plot, cmap=color_map, norm=LogNorm())\n        ax[\'B\'].set_title(secondary_title)\n    \n    ax[\'C\'].set_xlabel("Pixel Values")\n

In [ ]:
def compare_images(image, bias, debiased_image, dark, dedarked_image, flat, flat_corrected_image, vmin, vmax):
    
    fig, ax = plt.subplot_mosaic(
        '''
        AABB
        AABB
        CCDD
        CCDD
        EEFF
        EEFF
        GGHH
        GGHH
        ''',
        figsize = (14, 32)
    )

    # Hide the fake empty plots 
    ax['H'].set_visible(False)
    
    ax['A'].grid(False)
    ax['A'].set_title("Untouched Image")
    ax['A'].imshow(image, norm = LogNorm(vmin = vmin, vmax = vmax), cmap = color_map)

    ax['B'].grid(False)
    ax['B'].set_title("Master Bias")
    bias_median = np.median(bias)
    bias_iqr = iqr(bias)
    ax['B'].imshow(bias, norm = LogNorm(vmin = bias_median - 3 * bias_iqr, vmax = bias_median + 3 * bias_iqr), cmap = color_map)

    ax['C'].grid(False)
    ax['C'].set_title("Debiased Image")
    ax['C'].imshow(debiased_image, norm = LogNorm(vmin = vmin, vmax = vmax), cmap = color_map)

    ax['D'].grid(False)
    ax['D'].set_title("Master Dark")
    ax['D'].imshow(dark, norm = LogNorm(), cmap = color_map)

    ax['E'].grid(False)
    ax['E'].set_title("Dedarked Image")
    ax['E'].imshow(dedarked_image, norm = LogNorm(vmin = vmin, vmax = vmax), cmap = color_map)

    ax['F'].grid(False)
    ax['F'].set_title("Master Flat")
    flat_median = np.median(flat)
    flat_iqr = iqr(flat)
    ax['F'].imshow(flat, norm = LogNorm(vmin = flat_median - 3 * flat_iqr, vmax = flat_median + 3 * flat_iqr), cmap = color_map)
    
    ax['G'].grid(False)
    ax['G'].set_title("Flat Corrected Image")
    ax['G'].imshow(flat_corrected_image, norm = LogNorm(vmin = vmin, vmax = vmax), cmap = color_map)

    #ax['C'].set_xlabel("Pixel Values")
    #ax['C'].set_ylabel("Log(count)")
    
    #ax['C'].hist(image.flatten(), color="red", bins=500, log=True);
    #ax['C'].set_xlim(0, 65536)
    #ax['C'].set_ylim(0, 10)
    
    # save image
    #if save_fig:    fig.savefig(image_name);

    plt.show();

# Calibrating and Plotting Data

In [ ]:
# Grab image and create master calibration images
image, header = get_fits_data(os.path.join(data_path, image_path))
bias, dark, flat = create_masters(data_path, filter)

# Calibrate image by subtracting the bias and dark, then dividing by the flat
debiased, dedarked, flat_corrected = correct_image(image, bias, dark, flat)

FileNotFoundError: [Errno 2] No such file or directory: 'data\\480\\m81\\M81_Halpha_20230419_055757.fits'

In [ ]:
# Plot the images both before and after corrections were made
compare_images(image, bias, debiased, dark * 1800, dedarked, flat, flat_corrected, log_vmin, log_vmax)

# Aperture Photometry

In [ ]:
# Create an aperture and annulus for the object
aperture = EllipticalAperture(aperture_position, a = semiMajor, b = semiMinor, theta = aperture_theta)
annulus_aperture = EllipticalAnnulus(aperture_position, a_in = semiMajor * annulus_inner, a_out = semiMajor * annulus_outer, 
                                     b_in = semiMinor * annulus_inner, b_out = semiMinor * annulus_outer, theta = aperture_theta)

# Plot the object with its apertures
plt.imshow(flat_corrected, cmap = color_map, origin = 'lower', 
           norm = LogNorm(vmin = log_vmin, vmax = log_vmax))
aperture.plot(color = 'lightgreen', lw = 1.5, alpha = 1);
annulus_aperture.plot(color = 'red', lw = 1.5, alpha = 1);

NameError: name 'flat_corrected' is not defined

In [ ]:
# Do photometry on the aperture and annulus
aperture_phot = aperture_photometry(flat_corrected, aperture)
aperture_phot['aperture_sum'].info.format = '%.8g'
annulus_phot = aperture_photometry(flat_corrected, annulus_aperture)

# Find the count per area in the annulus to get the sky background
annulus_area = np.pi * semiMajor * semiMinor * ((annulus_outer ** 2) - (annulus_inner ** 2))
aperture_area = np.pi * aperture_a * aperture_b
annulus_count_per_m2 = annulus_phot['aperture_sum'] / annulus_area

# Subtract the sky background from the aperture photometry
sky_background = annulus_count_per_m2 * aperture_area
adjusted_counts = aperture_phot['aperture_sum'] - sky_background

# Get Star Formation Rate

In [ ]:
# Get the number of photons that hit in this area per second
electrons_per_sec = (adjusted_counts[0] * gain) / exposure
photons_per_sec = electrons_per_sec / quantum_efficiency

# Find the luminosity of the target
hc = 1.986E-25 # Speed of light times planck constant (in J/m)
mean_photon_energy = (hc / wavelength) # Average energy of a photon of this wavelength
flux_density = photons_per_sec * mean_photon_energy
luminosity = flux_density * 4 * np.pi * (distance ** 2)
luminosity *= 10000000 # Converts from watts to ergs/s
print(f"Luminosity: {luminosity} ergs/s")

# Use the luminosity in Halpha to calculate the star formation rate
SFR = 7.9 * (10 ** (-42)) * luminosity
print(f"SFR: {SFR:.2e} solar masses per year")